<a href="https://colab.research.google.com/github/Hamid-Mofidi/PNP/blob/main/Bifurcation_of_lambda/bif_of_lam_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is for the bifurcation analysis of flux ratio for studying the
effects of permanent charges on ionic flows

In [ ]:
from scipy.optimize import root
import numpy as np
import matplotlib.pyplot as plt

def equations(x):
    A, I, V, l = x

    k = 1
    r = 2

    sqA    = np.sqrt(1+A**2)
    sqB    = np.sqrt(1+(l-A+r)**2) #assume that alpha=1/3, beta=2/3
    logAB  = np.log(np.nan_to_num((l-A+r)*(sqA-1), nan = +1e-15) )- np.log(np.nan_to_num (A*(sqB-1), nan = +1e-15) )
    rho    = (A-l)**2 + (sqA - sqB)*(A-l)
    sigma  = l/r 
    logSBA = np.log(np.nan_to_num( sigma*(l-A+r) , nan = +1e-15) ) - np.log( np.nan_to_num(A, nan = +1e-15) ) # not a real number
    siglog = 3*sigma*np.log( np.nan_to_num(sigma, nan = +1e-15) )/(l*(sigma-1))

    numI1   = (( (-1)**(k+1) )*siglog) * (A-l)**2
    numI2   = (logAB - ((-1)**k) * np.log( np.nan_to_num(sigma, nan = +1e-15) ))*(A-l)
    gamma1  = 1/(I-(A-l) * sqA)
    gamma2  = 1/(I-(A-l) * sqB)
    M       = I * (gamma2 - gamma1)+ rho/I
    ABGamma = A * gamma1 + (l-A+r) * gamma2
    ABGammaR= (1/A) * gamma1 + (1/(l-A+r)) * gamma2
    G41     = ( 1- (A-l)*ABGamma )*( logSBA + (A-l) * siglog )
    G42     = ( 1- (A-l)*ABGamma )* M
    G43     = ( I+ ( ((-1)**k) * (A-l) ) ) * siglog * M
    GR      =( ( I**2-(A-l)**2 )/(A-l) ) * M * ABGammaR

  
    
    f1 = rho - (I* ( np.nan_to_num(np.log(np.nan_to_num( I-(A-l)*sqB, nan = +1e-10) ), nan = +1e-10)- np.nan_to_num(np.log(np.nan_to_num( I-(A-l)*sqA , nan = +1e-10) ) , nan = +1e-10) ) )
    f2 = V - logAB + ((I*logSBA-rho)/(A-l))
    f3 = I - (numI1 + numI2+rho)/(logSBA + (siglog)*(A-l))
    f4 = G41 - G42 + G43 - GR
    
    return [f1, f2, f3, f4]


# set a range of initial values
A_range = np.linspace(0, 2, 20)
I_range = np.linspace(0, 2, 20)
V_range = np.linspace(0, 10, 20)
l_range = np.linspace(0, 10, 20)

# initialize arrays to store solutions
A_sol = []
I_sol = []
V_sol = []
l_sol = []

# loop through all combinations of initial values
for a in A_range:
    for i in I_range:
        for v in V_range:
            for ll in l_range:
                x0 = [a, i, v, ll]
                sol = root(equations, x0, method='hybr')
                if sol.success: # check if solution was found
                    A_sol.append(sol.x[0])
                    I_sol.append(sol.x[1])
                    V_sol.append(sol.x[2])
                    l_sol.append(sol.x[3])

# plot the solutions
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].scatter(A_sol, I_sol)
axs[0, 0].set_xlabel('A')
axs[0, 0].set_ylabel('I')
axs[0, 1].scatter(A_sol, V_sol)
axs[0, 1].set_xlabel('A')
axs[0, 1].set_ylabel('V')
axs[1, 0].scatter(A_sol, l_sol)
axs[1, 0].set_xlabel('A')
axs[1, 0].set_ylabel('l')
axs[1, 1].scatter(I_sol, V_sol)
axs[1, 1].set_xlabel('I')
axs[1, 1].set_ylabel('V')
plt.show()

# x0 = [0.5, 0.5, 5, 5]
# sol = root(equations, x0, method='hybr')
# sol = root(equations, x0, method='broyden1')
# sol = root(equations, x0, method='anderson')

# print(sol.x)
